In [ ]:
import numpy as np
import open3d as o3d
from pykinect2 import PyKinectV2, PyKinectRuntime

In [ ]:
# Main Settings

# Angle between each scan in degrees
ANGLE_INCR: int = 30

# The target object type.
TARGET: str = 'sphere'

In [ ]:
# Initialize Kinect

kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth)

In [ ]:
# Initialize Pointclouds Container

pcds: list(o3d.geometry.Pointcloud) = []

In [ ]:
viz = o3d.visualization.Visualizer()
viz.create_window(window_name='KinectScan', width=720, height=480)

In [ ]:
for angle in range(0, 360, ANGLE_INCR):

    # Get Frame
    if kinect.has_new_color_frame() and kinect.has_new_depth_frame():

        color_frame = kinect.get_last_color_frame()
        depth_frame = kinect.get_last_depth_frame()

        # Create Pointcloud
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
            o3d.geometry.RGBDImage.create_from_color_and_depth(
                o3d.geometry.Image(color_frame),
                o3d.geometry.Image(depth_frame),
                depth_scale=1000,
                depth_trunc=1000,
                convert_rgb_to_intensity=False
            ),
            o3d.camera.PinholeCameraIntrinsic(
                o3d.camera.PinholeCameraIntrinsicParameters.Kinect2DepthCameraDefault
            )
        )

        kinect.release_last_color_frame()
        kinect.release_last_depth_frame()

        # Rotate Pointcloud

        pcd.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle([0, 1, 0], angle), center=[0, 0, 0])

        # Add Pointcloud to Container

        pcds.append(pcd)
        viz.add_geometry(pcd)

        # Update Visualization
        viz.update_renderer()
        viz.poll_events()

viz.destroy_window()

In [ ]:
# Register Collected Pointclouds

assert len(pcds) > 0

final_pcd: o3d.geometry.Pointcloud = pcds[0]
for pcd in pcds[1:]:
    icpRegResults = o3d.pipelines.registration.registration_icp(
        pcd, 
        final_pcd, 
        0.1, 
        np.identity(4), 
        o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000)
    )

    pcd.transform(icpRegResults.transformation)
    final_pcd += pcd

In [ ]:
# Visualize Result

o3d.visualization.draw_geometries([final_pcd])

In [ ]:
# Save Result

o3d.io.write_point_cloud(f'../data/Kinect/{TARGET}/result.ply', final_pcd)